In [1]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('bdu_clean.csv')  


In [3]:
df.drop(['month_name', 'high_wind_event', 'year'], axis=1, inplace=True)

In [4]:
encoder = OneHotEncoder(sparse_output=False)
encoded_array = encoder.fit_transform(df[['skyc1']])  # Output shape (n_samples, n_categories)
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['skyc1']))
df = pd.concat([df, encoded_df], axis=1)
df = df.drop(columns=['skyc1'], axis=1)

In [5]:
df['date'] = pd.to_datetime(df['date'])
df['timestamp'] = df['date'].astype(int) / 10**9 
df = df.drop(columns=['date'], axis=1)
df.columns

Index(['tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'p01i', 'alti', 'vsby', 'feel',
       'gust_factor', 'month', 'ext_gust_12h', 'skyc1_   ', 'skyc1_BKN',
       'skyc1_CLR', 'skyc1_FEW', 'skyc1_OVC', 'skyc1_SCT', 'skyc1_VV ',
       'timestamp'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['ext_gust_12h'], axis=1)
Y = df[['ext_gust_12h']]

In [7]:
### let's try xgboost

In [8]:
from imblearn.over_sampling import SMOTE 
oversample = SMOTE(random_state=1234, sampling_strategy=0.75)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.15)

X_train, y_train = oversample.fit_resample(X_train, y_train) 

In [9]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix as cmatrix
from sklearn.metrics import classification_report as creport

print(creport(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94     51889
           1       0.37      0.35      0.36      4771

    accuracy                           0.89     56660
   macro avg       0.65      0.65      0.65     56660
weighted avg       0.89      0.89      0.89     56660



In [11]:
#(tn, fp, fn, tp)
cmatrix(y_test, y_pred)

array([[49048,  2841],
       [ 3124,  1647]])

In [12]:
4720/(4720+51940)

0.0833039181080127

In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=200, class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
print(creport(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     51889
           1       0.00      0.00      0.00      4771

    accuracy                           0.92     56660
   macro avg       0.46      0.50      0.48     56660
weighted avg       0.84      0.92      0.88     56660



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
#(tn, fp, fn, tp)
cmatrix(y_test, y_pred)

array([[51889,     0],
       [ 4771,     0]])